<a href="https://colab.research.google.com/github/abdulR1806/BSS24-25/blob/main/Google_News_Scraping_v_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirements

In [ ]:
!pip install selenium gnews GoogleNews==1.6.13 free-proxy googlesearch-python

In [ ]:
# !apt-get update
# !apt-get install chromium-driver

In [ ]:
# ! export | grep  HTTPS_PROXY

fungsi untuk setup proxy dan chromedriver

In [ ]:
# from fp.fp import FreeProxy

# def get_proxy(country=['US', 'BR', 'ID']):
#   proxy = None

#   # Loop until we got proxy and proxy is different from before
#   while proxy is None:
#     try:
#       proxy = FreeProxy(country_id=country, timeout=0.5, https=True, rand=True).get()
#     except:
#       pass

#   return proxy

# def check_proxy(proxy, list_proxy):

#   if proxy == None:
#     proxy = get_proxy()
#   else:
#     while proxy in list_proxy:
#       proxy = get_proxy()

#   list_proxy.append(proxy)
#   return proxy

In [ ]:
# from selenium import webdriver

# def web_driver():
#   chrome_options = webdriver.ChromeOptions()
#   chrome_options.add_argument('--headless')
#   chrome_options.add_argument('--no-sandbox')
#   chrome_options.add_argument('--disable-dev-shm-usage')
#   chrome_options.add_argument('--disable-gpu')
#   chrome_options.add_argument('--verbose')
#   chrome_options.add_argument('--window-size=1920,1200')
#   chrome_options.add_argument('--disable-extensions')
#   driver = webdriver.Chrome( options=chrome_options)
#   return driver

In [ ]:
import logging
import time
from google.colab import files
from gnews import GNews
from GoogleNews import GoogleNews
import random
import pandas as pd
import numpy as np
import datetime
import pytz

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Scraping Function

In [ ]:
def get_data(id: str, gid: str) -> pd.DataFrame:
    """
    Fetches data from a specified Google Sheets document and sheet as a pandas DataFrame.

    This function constructs a URL to access a Google Sheets document in CSV format by using
    the document's ID and the specific sheet's GID. It then reads the CSV data into a pandas DataFrame
    and returns this DataFrame.

    Parameters:
    - id (str): The unique identifier for the Google Sheets document.
    - gid (str): The unique identifier for the specific sheet within the document.

    Returns:
    - pd.DataFrame: The data from the specified Google Sheet and sheet as a pandas DataFrame.

    Example:
    >>> data_df = get_data('1a2B3cD4E5fG6hI7J8K9L0MnOpQrStUvW', '1234567890')
    # This would fetch the data from the Google Sheet with the provided ID and GID.

    Note:
    - Ensure that the Google Sheet is publicly accessible or shared with the client service account
      if using Google Cloud authentication, to avoid access denied errors.
    """
    if not id or not gid:
        raise ValueError("Google Sheet ID and GID must be provided.")

    url = f"https://docs.google.com/spreadsheets/d/{id}/export?format=csv&gid={gid}"
    try:
        data = pd.read_csv(url)
    except Exception as e:
        logging.error(f"Failed to fetch data from Google Sheets: {e}")
        raise
    return data

def transform_keyword(all_debitur: pd.DataFrame, keyword_filter: pd.DataFrame, filter_ket: bool=False) -> dict:
    """
    Transforms the `all_debitur` DataFrame by merging it with `keyword_filter` DataFrame,
    and modifies the 'Name & Company' column by appending the 'Filter Keyword'.

    Args:
        all_debitur (pd.DataFrame): A DataFrame containing debtor information.
        keyword_filter (pd.DataFrame): A DataFrame containing filter keywords associated with segments.

    Returns:
        dict: A dictionary where the keys are unique segments and the values are DataFrames
              filtered by each segment.
    """
    # Validate input DataFrames
    required_columns_debitur = {'Segment', 'Name & Company'}
    required_columns_filter = {'Segment', 'Filter Keyword'}

    if not required_columns_debitur.issubset(all_debitur.columns):
        error_msg = f"all_debitur DataFrame must contain columns: {required_columns_debitur}"
        logging.error(error_msg)
        raise ValueError(error_msg)

    if not required_columns_filter.issubset(keyword_filter.columns):
        error_msg = f"keyword_filter DataFrame must contain columns: {required_columns_filter}"
        logging.error(error_msg)
        raise ValueError(error_msg)

    # Merge the two DataFrames on the 'Segment' column, filling missing values with a space
    combined_deb = pd.merge(all_debitur, keyword_filter, how="outer", on=["Segment"]).fillna("")

    if combined_deb.empty:
        warning_msg = "The merged DataFrame is empty. Check input DataFrames for proper data."
        logging.warning(warning_msg)

    # Transform 'Name & Company' column
    combined_deb['Name & Company'] = '"' + combined_deb['Name & Company'] + '"'
    combined_deb['Name & Company'] = combined_deb['Name & Company'] + " " + combined_deb['Filter Keyword']
    if filter_ket:
      combined_deb['Name & Company'] = combined_deb['Name & Company'] + " " + combined_deb['Keterangan']

    # Create a dictionary with unique segments as keys and filtered DataFrames as values
    segment_dict = {segment: combined_deb[combined_deb.Segment == segment]["Name & Company"].to_list() for segment in combined_deb.Segment.unique()}

    return segment_dict

In [ ]:
def transform_news_data(result_data: dict, type_deb: str, filter: bool = False, keyword_period: str = "") -> pd.DataFrame:
    """
    Transforms the news data from JSON format to a pandas DataFrame and applies optional filtering.

    This function processes a dictionary of news data, normalizes it into a DataFrame, and optionally filters
    the data based on a specified keyword period. It also ensures URL formats are corrected and columns are reordered.

    Parameters:
    - result_data (dict): Dictionary containing news data with keywords as keys.
    - type_deb (str): Type of the entity being checked.
    - filter (bool, optional): Whether to apply filtering on the 'date' column (default is False).
    - keyword_period (str, optional): The keyword period for filtering the 'date' column (default is "").

    Returns:
    - pd.DataFrame: The transformed and optionally filtered news data as a pandas DataFrame.
    """
    data = []
    for debitur, news_items in result_data.items():
        df = pd.json_normalize(news_items)
        df["keyword"] = debitur
        data.append(df)

    if not data:
        logging.warning("No data to transform.")
        return pd.DataFrame(columns=['keyword', 'title', 'date', 'link', 'media'])

    result = pd.concat(data, ignore_index=True)

    try:
        result['link'] = result['link'].apply(lambda x: x.split("&ved=")[0])
    except KeyError as e:
        logging.error(f"Error processing links: {e}")
        result = result.rename(columns={"title": "title", "url": "link", "published date": "date", "publisher.title": "media"}, errors="raise")
        result['link'] = result['link'].apply(lambda x: x.split("&ved=")[0])

    cols = result.columns.tolist()
    result = result[cols[-1:] + cols[:-1]]

    if filter:
        result = result[result['date'].str.contains(keyword_period)]
        result = result[~result['date'].str.contains("2023")]

    result = result[['keyword', 'title', 'date', 'link', 'media']]
    result = result.drop_duplicates(subset=['title'])
    return result

def create_excel_news(type_deb: str, df_dict: dict) -> str:
    """
    Creates an Excel file from the provided dataframes and returns the filename.

    This function exports the cleaned and processed data to an Excel file with sheets named according to
    the keys in the provided dictionary. The filename includes the current date and time to ensure uniqueness.

    Parameters:
    - type_deb (str): The type of entity being checked.
    - df_dict (dict): Dictionary containing dataframes to be written to Excel, with keys as sheet names.

    Returns:
    - str: The filename of the created Excel file.
    """
    now = datetime.datetime.now(pytz.timezone('Asia/Jakarta'))
    name_file = f'news_{type_deb}_{now.strftime("%Y-%m-%d %H:%M:%S")}.xlsx'

    try:
        with pd.ExcelWriter(name_file) as writer:
            for name, df in df_dict.items():
                df.to_excel(writer, sheet_name=name, index=False)
    except Exception as e:
        logging.error(f"Failed to create Excel file: {e}")
        raise

    return name_file

def web_checking(period_check: str, type_deb: str, keyword_list: list, time_sleep: int, check_wholesale: bool = False, start: int = 1, end: int = None) -> pd.DataFrame:
    """
    Performs a web check for news articles and search results based on given keywords.

    This function uses Google News and Google Search to fetch news articles and search results
    for a list of keywords. It then processes the results, logs the search activities, and
    saves the results into an Excel file.

    Parameters:
    - period_check (str): The time period to check for news articles (e.g., '14d' for 14 days).
    - type_deb (str): The type of entity being checked (e.g., 'esme').
    - keyword_list (list): A list of keywords to search for.
    - time_sleep (int): The time to sleep between searches in seconds.
    - check_wholesale (bool, optional): Whether to perform a wholesale check (default is False).
    - start (int, optional): The starting index for the keyword list (default is 1).
    - end (int, optional): The ending index for the keyword list (default is None, which means the end of the list).

    Returns:
    - pd.DataFrame: A DataFrame containing the combined results from the searches.
    """
    all_news = {}
    all_search = {}
    list_deb = []
    total_news = []

    googlenews = GNews(language='id', country='ID', period=period_check, max_results=100)
    googlesearch = GoogleNews(lang='ID', period=period_check, region="ID", encode='utf-8')

    start_list = start - 1
    end_list = end - 1 if end else len(keyword_list) - 1

    for i in range(start_list, end_list + 1):
        debitur = keyword_list[i]
        news = googlenews.get_news(debitur)

        if news:
            all_news[debitur] = news
            type_source = "google news"
        else:
            googlesearch.clear()
            googlesearch.search(debitur)
            news = googlesearch.results(sort=True)
            all_search[debitur] = news
            type_source = "google search"

            if time_sleep < 10 or len(news) >= 10:
                try:
                    random_time = random.randint(1, len(news)) + 10
                    time.sleep(random_time)
                except:
                    time.sleep(10)

        list_deb.append(debitur)
        total_news.append(len(news))
        print(f"{i+1} - {type_deb} :: berita {debitur} sudah ditambahkan dari {type_source}. Jumlah berita: {len(news)}")
        time.sleep(time_sleep)

    log_data = {"type_deb": type_deb, "keyword": list_deb, "hasil_pencarian": total_news}
    log_df = pd.DataFrame(log_data)

    try:
        df_news = transform_news_data(all_news, type_deb=type_deb, filter=False)
        df_news['source'] = "google_news"
    except Exception as e:
        logging.error(f"Failed to transform news data: {e}")
        df_news = pd.DataFrame(columns=['keyword', 'title', 'date', 'link', 'media'])

    try:
        df_search = transform_news_data(all_search, type_deb=type_deb, filter=not check_wholesale, keyword_period="0 bulan|minggu|hari|kemarin|jam|menit|Yesterday|ago")
        df_search['source'] = "google_search"
    except Exception as e:
        logging.error(f"Failed to transform search data: {e}")
        df_search = pd.DataFrame(columns=['keyword', 'title', 'date', 'link', 'media'])

    final_result = pd.concat([df_news, df_search], ignore_index=True)
    final_result["Type"] = type_deb

    final_dict = {"Web Checking": final_result, "Logs": log_df}

    try:
        file_excel = create_excel_news(type_deb, final_dict)
        files.download(file_excel)
        print(f"File excel sudah dibuat untuk debitur {type_deb}")
    except Exception as e:
        logging.error(f"Failed to create or download Excel file: {e}")
        raise

    return final_result

Fungsi untuk legal checking

In [ ]:
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_table_case(table: str, period: str) -> pd.DataFrame:
    """
    Processes the HTML table into a pandas DataFrame and filters it based on the given period.

    Parameters:
    - table (str): HTML content of the table.
    - period (str): Date string to filter the cases.

    Returns:
    - pd.DataFrame: Filtered DataFrame containing case information.
    """
    df_list = pd.read_html(table, header=0, extract_links="body")
    df = df_list[0]
    df.columns = df.columns.droplevel(1)

    # Extract data and links separately
    df = df.applymap(lambda x: x[0] if isinstance(x, tuple) else x)
    df["Link"] = df_list[0]["Link"].apply(lambda x: x[1] if isinstance(x, tuple) else '')

    # Convert 'Tanggal Register' to datetime and filter based on period
    df["Tanggal Register"] = pd.to_datetime(df["Tanggal Register"], format='%d %b %Y')
    df.set_index("No", inplace=True)
    filtered_df = df.query(f"`Tanggal Register` > '{period}'")

    return filtered_df

def get_total_case(url: str, keyword: str, period: str):
    """
    Fetches the total number of cases and the detailed case data based on the keyword and period.

    Parameters:
    - url (str): The URL to fetch data from.
    - keyword (str): The keyword to search for.
    - period (str): The period to filter the cases.

    Returns:
    - tuple: Total number of cases and a DataFrame of the case details.
    """
    driver = web_driver()  # Ensure you have the correct WebDriver path configured
    keyword = keyword.replace('"', '').replace("'", '')

    try :
      driver.get(url)
      search_box = driver.find_element(By.XPATH, '//input[@name="search_keyword"]')
      search_box.send_keys(keyword)
      search_box.submit()

      result_text = driver.find_element(By.XPATH, '//div[@class="total_perkara"]').text
      total_perkara = int(re.search(r'Total : (\d+)', result_text).group(1))
    except Exception as e:
        logging.error(f"Error fetching webpage for link '{url}': {e}")

    if total_perkara > 0:
        table_html = driver.find_element(By.XPATH, '//table[@id="tablePerkaraAll"]').get_attribute('outerHTML')
        data = get_table_case(table=table_html, period=period)
        print(f"{url} :: {keyword} :: {data.shape[0]} results from {total_perkara} cases")

        return total_perkara, data
    else:
        print(f"{url} :: {keyword} :: 0 results from {total_perkara} cases")

        empty_df = pd.DataFrame(columns=['Nomor Perkara', 'Tanggal Register', 'Klasifikasi Perkara', 'Para Pihak', 'Status Perkara', 'Lama Proses', 'Link'])
        return total_perkara, empty_df

def legal_checking(period_filter: str, list_url_sipp: list, keyword_list: list, time_sleep: int) -> pd.DataFrame:
    """
    Performs a legal check for cases based on given keywords and period.

    Parameters:
    - period_filter (str): The date period to filter the cases.
    - url (str): The URL to fetch data from.
    - keyword_list (list): A list of keywords to search for.
    - time_sleep (int): The time to sleep between searches in seconds.

    Returns:
    - pd.DataFrame: A DataFrame containing the combined results from the searches.
    """
    all_df = []
    len_df = []
    list_url = []
    total_perkara = []
    list_keyword = []

    for url in list_url_sipp:

      for keyword in keyword_list:
          try:
              total, data = get_total_case(url=url, keyword=keyword, period=period_filter)
          except Exception as e:
              logging.error(f"Error fetching data for keyword '{keyword}': {e}")
              continue

          list_url.append(url)
          list_keyword.append(keyword)
          total_perkara.append(total)
          all_df.append(data)
          len_df.append(data.shape[0])

          time.sleep(time_sleep)

    # Create log DataFrame
    log_data = {"url": list_url, "keyword": list_keyword, "hasil pencarian": total_perkara, "hasil filter": len_df}
    log_df = pd.DataFrame(log_data)

    # Concatenate all dataframes in the 'data' list into a single dataframe
    final_result = pd.concat(all_df, ignore_index=True)
    final_dict = {"Legal Checking": final_result, "Logs": log_df}

    try:
        file_excel = create_excel_news(type_deb="legal_checking", df_dict=final_dict)
        files.download(file_excel)
        print("File excel sudah dibuat untuk legal checking debitur")
    except Exception as e:
        logging.error(f"Failed to create or download Excel file: {e}")
        raise

    return final_result

In [ ]:
list_proxy = []
proxy = None

# Web Checking & Legal Checking per Debitur

Link Untuk Menambahkan kata kunci debitur : https://docs.google.com/spreadsheets/d/1fyKw095hZ6OSJeg0ZVN_lbUF1qlT2MHjNEOJXRVjTIk/edit?gid=0#gid=0

In [ ]:
id_sheet = "1fyKw095hZ6OSJeg0ZVN_lbUF1qlT2MHjNEOJXRVjTIk"

all_debitur = get_data(id=id_sheet, gid="0")
keyword_filter = get_data(id=id_sheet, gid = "395481292")

dict_all_debtors = transform_keyword(all_debitur, keyword_filter)

In [ ]:
dict_all_debtors.keys()

Lakukan Legal Checking

In [ ]:
# all_debitur = get_data(id=id_sheet, gid="0")

# list_wholesale_legal = all_debitur.loc[all_debitur.Segment=="Wholesale"]['Name & Company'].tolist()
# list_sme_legal = all_debitur.loc[all_debitur.Segment=="SME"]['Name & Company'].tolist()
# list_esme_legal = all_debitur.loc[all_debitur.Segment=="ESME"]['Name & Company'].tolist()
# list_fi_legal = all_debitur.loc[all_debitur.Segment=="FI"]['Name & Company'].tolist()

In [ ]:
# period_filter="2024-11-01"
# sipp_urls = ["https://sipp.pn-jakartaselatan.go.id/list_perkara/search", "https://sipp.pn-jakartabarat.go.id/list_perkara/search","https://sipp.pn-jakartatimur.go.id/list_perkara/search","https://sipp.pn-jakartautara.go.id/list_perkara/search"]
# # sipp_urls = ["https://sipp.pn-jakartaselatan.go.id/list_perkara/search", "https://sipp.pn-jakartabarat.go.id/list_perkara/search","https://www.pn-negara.go.id/","https://www.mahkamahagung.go.id/id"
# time_sleep = 10
# result = []

In [ ]:
# print("##############Legal Checking untuk Debitur Wholesale##############")
# wholesale_result = legal_checking(period_filter=period_filter, list_url_sipp=sipp_urls, keyword_list=list_wholesale_legal, time_sleep=time_sleep)
# time.sleep(120)
# print("##############Legal Checking untuk Debitur SME##############")
# sme_result = legal_checking(period_filter=period_filter, list_url_sipp=sipp_urls, keyword_list=list_sme_legal, time_sleep=time_sleep)
# time.sleep(120)
# print("##############Legal Checking untuk Debitur ESME##############")
# esme_result = legal_checking(period_filter=period_filter, list_url_sipp=sipp_urls, keyword_list=list_esme_legal, time_sleep=time_sleep)
# time.sleep(120)
# print("##############Legal Checking untuk Debitur FI##############")
# fi_result = legal_checking(period_filter=period_filter, list_url_sipp=sipp_urls, keyword_list=list_fi_legal, time_sleep=time_sleep)

Legal Checking per Debitur

In [ ]:
# legal_checking(period_filter=period_filter, list_url_sipp=sipp_urls, keyword_list=["Santoso Halim","Sukoco Halim"], time_sleep=time_sleep)

Lakukan Web Checking

In [ ]:
# Web Checking
df_esme = web_checking(period_check="14d", type_deb = "ESME", keyword_list=dict_all_debtors["ESME"], start=1, end=50, time_sleep=30)

In [ ]:
# Web Checking
time.sleep(300)
df_esme = web_checking(period_check="14d", type_deb = "ESME", keyword_list=dict_all_debtors["ESME"], start=51, time_sleep=30)

In [ ]:
time.sleep(300)
df_fi = web_checking(period_check="14d", type_deb = "FI", keyword_list=dict_all_debtors["FI"], start=1, end=50, time_sleep=30)

In [ ]:
time.sleep(300)
df_fi = web_checking(period_check="14d", type_deb = "FI", keyword_list=dict_all_debtors["FI"], start=51, time_sleep=30)

Jika sudah selesai, Disconnect Notebook kemudian connect lagi dan run dari awal

In [ ]:
time.sleep(300) #sleep antar baris
df_sme = web_checking(period_check="14d", type_deb = "SME", keyword_list=dict_all_debtors["SME"], start=1, time_sleep=30) #sleep antar keyword

In [ ]:
time.sleep(300)
df_sme = web_checking(period_check="14d", type_deb = "SME", keyword_list=dict_all_debtors["SME"], start=51, time_sleep=30)

In [ ]:
time.sleep(300)
df_wholesale = web_checking(period_check="14d", type_deb = "WHOLESALE", keyword_list=dict_all_debtors["WHOLESALE"], start=1, end=50, time_sleep=30)

In [ ]:
time.sleep(300)
df_wholesale = web_checking(period_check="14d", type_deb = "WHOLESALE", keyword_list=dict_all_debtors["WHOLESALE"], start=51, time_sleep=30)